<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#4.-Inference" data-toc-modified-id="4.-Inference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>4. Inference</a></span><ul class="toc-item"><li><span><a href="#4-1.-(just-for-test)-Data-Preparation" data-toc-modified-id="4-1.-(just-for-test)-Data-Preparation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>4-1. (just for test) Data Preparation</a></span></li></ul></li></ul></div>

# 4. Inference

In [1]:
import sys, os
sys.path.append("../../..")

import pathSetting
import pandas as pd
from KETIToolDL.CLUSTTool.common import p1_integratedDataSaving as p1
from KETIToolDL.CLUSTTool.common import p2_dataSelection as p2
from KETIToolDL.CLUSTTool.common import p4_testing as p4
from KETIToolDL.CLUSTTool.Classification import p4_testing as p4C

import torch
#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cuda is available.


## 4-1. (just for test) Data Preparation 

In [2]:
# 0. pick only one data
DataMeta = p1.readJsonData(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())
# dataX

dataName_X = dataList[2]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[3]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

windowNum =128
dataX = p2.getSavedIntegratedData(dataSaveMode_X, dataName_X, dataFolderPath)[:windowNum]
datay = p2.getSavedIntegratedData(dataSaveMode_y, dataName_y, dataFolderPath)[:1]
from Clust.clust.transformation.type.DFToNPArray import transDFtoNP
inputX, inputy = transDFtoNP(dataX, datay, windowNum)
print(inputX, inputy) # Real Input

[[[ 0.01165315  0.01310909  0.01126885 ...  0.00129151  0.00147
    0.00257384]
  [-0.02939904 -0.03972867 -0.05240586 ...  0.01173502  0.00951741
    0.00730507]
  [ 0.1068262   0.1524549   0.2168462  ...  0.00366559  0.00404194
    0.00488844]
  ...
  [ 1.041216    1.041803    1.039086   ...  0.9930164   0.9932414
    0.9943906 ]
  [-0.2697959  -0.280025   -0.2926631  ... -0.2599865  -0.2620643
   -0.2641348 ]
  [ 0.02377977  0.07629271  0.1474754  ...  0.1443951   0.1447033
    0.1454939 ]]] [4.]


In [3]:
# 1 Read Parameter and model information
ModelMeta =p1.readJsonData(pathSetting.trainModelMetaFilePath)
modelList = list(ModelMeta.keys())
modelName = modelList[0]

X_scalerFilePath = ModelMeta[modelName]['files']['XScalerFile']["filePath"]
y_scalerFilePath = ModelMeta[modelName]['files']['yScalerFile']["filePath"]
modelFilePath = ModelMeta[modelName]['files']['modelFile']["filePath"]
    
featureList = ModelMeta[modelName]["featureList"]
target = ModelMeta[modelName]["target"]
scalerParam = ModelMeta[modelName]["scalerParam"]
model_method = ModelMeta[modelName]["model_method"]
trainParameter = ModelMeta[modelName]["trainParameter"]

In [4]:
#2 ## ???
#from Clust.clust.transformation.type.NPArrayToDF import trans3NPtoDF
#inputXDF, y = trans3NPtoDF(inputX, inputy,'2000-1-1')

In [5]:
# 3. Scaling
test_x, scaler_X = p4.getScaledTestData(dataX[featureList], X_scalerFilePath, scalerParam)
test_y, scaler_y = p4.getScaledTestData(datay[target], y_scalerFilePath, scalerParam)
# y  값은 더미 값이 들어가는 것임

In [6]:
# 4. Testing
batch_size=1

df_result, result_metrics_df, acc = p4C.getResultMetrics(test_x, test_y, model_method, target, modelFilePath, scalerParam, scaler_y, trainParameter, batch_size, device)

/home/hwangjisoo/바탕화면/Clust/KETIAppTestCode/MLModelTest/ClassificationModelTest/../../../KETIPreDataTransformation/dataType/DFToNPArray.py:37: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
/home/hwangjisoo/바탕화면/Clust/KETIAppTestCode/MLModelTest/ClassificationModelTest/../../../KETIPreDataTransformation/dataType/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]



Start testing data

** Performance of test dataset ==> PROB = [[9.5148051e-01 4.6992615e-02 3.5871699e-04 3.3532301e-04 5.3666160e-04
  2.9622717e-04]], ACC = 1.0
** Dimension of result for test dataset = (1,)
scale
** Performance of test dataset (After Inverse Scaling)==> MSE = 0.0, MAE = 0.0


In [7]:
df_result

,value,prediction
0,0.0,0.0


In [8]:
result_metrics_df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,1.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,1.0
weighted avg,1.0,1.0,1.0,1.0


In [9]:
ModelMeta[modelName]['files']['modelFile']["filePath"]

['./Models/LSTM_cf/CLUST/ActionPattern/afc5bf792ed6f59e371a84e15aca5813_5859aaa1977f607dd6decedede00df0d/f2a36ead1ca2fef043aa1df6ee3786c1/model.pt']